In [1]:
import os
import pandas as pd
import numpy as np
import mygene
import gseapy as gp
import gseapy.plot as gseaplot
import matplotlib.pyplot as plt

from gseapy import enrichment_map
import networkx as nx

# %matplotlib inline
# %config InlineBackend.figure_format='retina' # mac
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt

from gseapy import Biomart
bm = Biomart()

from gseapy import gseaplot, heatmap

In [2]:
def enrich_genes(top_ranked_genes):

    enr_Reactome_up = gp.enrichr(gene_list=top_ranked_genes,
                                 gene_sets=['Reactome_2022'],
                                 organism='Human', 
                                 cutoff=0.5)
    
    enr_GOBP_up = gp.enrichr(gene_list=top_ranked_genes,
                             gene_sets=['GO_Biological_Process_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    enr_GOMF_up = gp.enrichr(gene_list=top_ranked_genes ,
                             gene_sets=['GO_Molecular_Function_2021'],
                             organism='Human', 
                             cutoff=0.5)
    
    
    # Write results to CSV files
    #file_name = os.path.basename(file_path).split('.')[0]
    #enr_Reactome_up.results.to_csv(f'{file_name}_enr_Reactome.csv')
    
    return enr_Reactome_up.results, enr_GOBP_up.results, enr_GOMF_up.results

In [26]:
targets4 = pd.read_csv('/Users/sofialuk/Desktop/mirna_shit/targets4.csv')
targets12 = pd.read_csv('/Users/sofialuk/Desktop/mirna_shit/targets12.csv')
targets24 = pd.read_csv('/Users/sofialuk/Desktop/mirna_shit/targets24.csv')

In [27]:
targets12

,Unnamed: 0,mirna,symbol
0,77940,hsa-miR-135b-5p,MRPS12
1,77941,hsa-miR-135b-5p,KCNAB3
2,77942,hsa-miR-135b-5p,KCND1
3,77943,hsa-miR-135b-5p,BMAL1
4,77944,hsa-miR-135b-5p,ZBTB44
...,...,...,...
13814,330065,hsa-miR-4299,SLC22A23
13815,330066,hsa-miR-4299,TET1
13816,330067,hsa-miR-4299,LSM14A
13817,330068,hsa-miR-4299,L3MBTL3


In [28]:
mirnas = list(set(set(targets12['mirna'])))

In [29]:
mirnas[1]

'hsa-miR-4293'

In [30]:
targets12[targets12['mirna'].str.contains(mirnas[0])]

,Unnamed: 0,mirna,symbol
8067,287226,hsa-miR-374a-5p,AHSA2P
8068,287227,hsa-miR-374a-5p,PDE4B
8069,287228,hsa-miR-374a-5p,TENT2
8070,287229,hsa-miR-374a-5p,ELOVL7
8071,287230,hsa-miR-374a-5p,SRSF7
...,...,...,...
8937,288096,hsa-miR-374a-5p,LEPR
8938,288097,hsa-miR-374a-5p,RAP1B
8939,288098,hsa-miR-374a-5p,HAPLN1
8940,288099,hsa-miR-374a-5p,STEAP2


In [31]:
miRNA_targets = {}

# Iterate over each miRNA and find the corresponding targets
for mirna in range(len(mirnas)):
    # Filter the DataFrame based on the miRNA number in the target gene column
                   
    targets = targets12[targets12['mirna'].str.contains(mirnas[mirna])]
    miRNA_targets[f'miR-{mirnas[mirna]}'] = targets

# Create a Pandas Series from the miRNA_targets dictionary
miRNA_targets

{'miR-hsa-miR-374a-5p':       Unnamed: 0            mirna  symbol
 8067      287226  hsa-miR-374a-5p  AHSA2P
 8068      287227  hsa-miR-374a-5p   PDE4B
 8069      287228  hsa-miR-374a-5p   TENT2
 8070      287229  hsa-miR-374a-5p  ELOVL7
 8071      287230  hsa-miR-374a-5p   SRSF7
 ...          ...              ...     ...
 8937      288096  hsa-miR-374a-5p    LEPR
 8938      288097  hsa-miR-374a-5p   RAP1B
 8939      288098  hsa-miR-374a-5p  HAPLN1
 8940      288099  hsa-miR-374a-5p  STEAP2
 8941      288100  hsa-miR-374a-5p   CELF4
 
 [875 rows x 3 columns],
 'miR-hsa-miR-4293':        Unnamed: 0         mirna   symbol
 11939      328190  hsa-miR-4293    KPNA1
 11940      328191  hsa-miR-4293  SLC30A6
 11941      328192  hsa-miR-4293   LGALSL
 11942      328193  hsa-miR-4293     AQP3
 11943      328194  hsa-miR-4293  CAPRIN2
 ...           ...           ...      ...
 12139      328390  hsa-miR-4293  ABHD16A
 12140      328391  hsa-miR-4293    RAP1B
 12141      328392  hsa-miR-4293   V

In [32]:
mirnas = list(miRNA_targets.keys())

In [33]:
# Filter out the keys with empty Series values

miRNA_targets_filtered = {k: v for k, v in miRNA_targets.items() if not v.empty}

In [34]:
miRNA_targets_filtered

{'miR-hsa-miR-374a-5p':       Unnamed: 0            mirna  symbol
 8067      287226  hsa-miR-374a-5p  AHSA2P
 8068      287227  hsa-miR-374a-5p   PDE4B
 8069      287228  hsa-miR-374a-5p   TENT2
 8070      287229  hsa-miR-374a-5p  ELOVL7
 8071      287230  hsa-miR-374a-5p   SRSF7
 ...          ...              ...     ...
 8937      288096  hsa-miR-374a-5p    LEPR
 8938      288097  hsa-miR-374a-5p   RAP1B
 8939      288098  hsa-miR-374a-5p  HAPLN1
 8940      288099  hsa-miR-374a-5p  STEAP2
 8941      288100  hsa-miR-374a-5p   CELF4
 
 [875 rows x 3 columns],
 'miR-hsa-miR-4293':        Unnamed: 0         mirna   symbol
 11939      328190  hsa-miR-4293    KPNA1
 11940      328191  hsa-miR-4293  SLC30A6
 11941      328192  hsa-miR-4293   LGALSL
 11942      328193  hsa-miR-4293     AQP3
 11943      328194  hsa-miR-4293  CAPRIN2
 ...           ...           ...      ...
 12139      328390  hsa-miR-4293  ABHD16A
 12140      328391  hsa-miR-4293    RAP1B
 12141      328392  hsa-miR-4293   V

In [35]:
miRNA_targets_filtered.keys()

dict_keys(['miR-hsa-miR-374a-5p', 'miR-hsa-miR-4293', 'miR-hsa-miR-4291', 'miR-hsa-miR-194-5p', 'miR-hsa-miR-4294', 'miR-hsa-miR-29a-3p', 'miR-hsa-miR-4296', 'miR-hsa-miR-4298', 'miR-hsa-miR-429', 'miR-hsa-miR-330-5p', 'miR-hsa-miR-4295', 'miR-hsa-miR-424-5p', 'miR-hsa-miR-135b-5p', 'miR-hsa-miR-155-3p', 'miR-hsa-miR-4292', 'miR-hsa-miR-16-5p', 'miR-hsa-miR-141-3p', 'miR-hsa-miR-335-5p', 'miR-hsa-miR-27b-3p', 'miR-hsa-miR-29c-3p', 'miR-hsa-miR-4290', 'miR-hsa-miR-27a-3p', 'miR-hsa-miR-200b-3p', 'miR-hsa-miR-4299', 'miR-hsa-miR-4297', 'miR-hsa-miR-21-5p'])

In [36]:
for curr_mirna in list(miRNA_targets_filtered.keys()):
    # Call the enrich_genes function and store the output in variables with dynamic names
    enr_Reactome_up, enr_GOBP_up, enr_GOMF_up = enrich_genes(miRNA_targets[curr_mirna]['symbol'].dropna())
    
    # Store the enriched gene lists in the globals() dictionary
    enr_Reactome_up_key = f"enr_Reactome_up_{curr_mirna.replace('-', '_')}"
    globals()[enr_Reactome_up_key] = enr_Reactome_up
    result_dict[curr_mirna] = globals()[enr_Reactome_up_key][:20]
    
print(result_dict)

{'miR-hsa-let-7a-3p':          Gene_set                                               Term  \
0   Reactome_2022                   Signal Transduction R-HSA-162582   
1   Reactome_2022  Signaling By Rho GTPases, Miro GTPases And RHO...   
2   Reactome_2022              Signaling By Rho GTPases R-HSA-194315   
3   Reactome_2022                       Circadian Clock R-HSA-400253   
4   Reactome_2022        Gene Expression (Transcription) R-HSA-74160   
5   Reactome_2022  Signaling By Receptor Tyrosine Kinases R-HSA-9...   
6   Reactome_2022                       Heme Signaling R-HSA-9707616   
7   Reactome_2022                     RHO GTPase Cycle R-HSA-9012999   
8   Reactome_2022  Constitutive Signaling By NOTCH1 t(7;9)(NOTCH1...   
9   Reactome_2022                  Signaling By NOTCH1 R-HSA-1980143   
10  Reactome_2022           Vesicle-mediated Transport R-HSA-5653656   
11  Reactome_2022  SUMO E3 Ligases SUMOylate Target Proteins R-HS...   
12  Reactome_2022                  Membran

In [37]:
miRNA_targets_filtered.keys()

dict_keys(['miR-hsa-miR-374a-5p', 'miR-hsa-miR-4293', 'miR-hsa-miR-4291', 'miR-hsa-miR-194-5p', 'miR-hsa-miR-4294', 'miR-hsa-miR-29a-3p', 'miR-hsa-miR-4296', 'miR-hsa-miR-4298', 'miR-hsa-miR-429', 'miR-hsa-miR-330-5p', 'miR-hsa-miR-4295', 'miR-hsa-miR-424-5p', 'miR-hsa-miR-135b-5p', 'miR-hsa-miR-155-3p', 'miR-hsa-miR-4292', 'miR-hsa-miR-16-5p', 'miR-hsa-miR-141-3p', 'miR-hsa-miR-335-5p', 'miR-hsa-miR-27b-3p', 'miR-hsa-miR-29c-3p', 'miR-hsa-miR-4290', 'miR-hsa-miR-27a-3p', 'miR-hsa-miR-200b-3p', 'miR-hsa-miR-4299', 'miR-hsa-miR-4297', 'miR-hsa-miR-21-5p'])

In [38]:
result_dict

{'miR-hsa-let-7a-3p':          Gene_set                                               Term  \
 0   Reactome_2022                   Signal Transduction R-HSA-162582   
 1   Reactome_2022  Signaling By Rho GTPases, Miro GTPases And RHO...   
 2   Reactome_2022              Signaling By Rho GTPases R-HSA-194315   
 3   Reactome_2022                       Circadian Clock R-HSA-400253   
 4   Reactome_2022        Gene Expression (Transcription) R-HSA-74160   
 5   Reactome_2022  Signaling By Receptor Tyrosine Kinases R-HSA-9...   
 6   Reactome_2022                       Heme Signaling R-HSA-9707616   
 7   Reactome_2022                     RHO GTPase Cycle R-HSA-9012999   
 8   Reactome_2022  Constitutive Signaling By NOTCH1 t(7;9)(NOTCH1...   
 9   Reactome_2022                  Signaling By NOTCH1 R-HSA-1980143   
 10  Reactome_2022           Vesicle-mediated Transport R-HSA-5653656   
 11  Reactome_2022  SUMO E3 Ligases SUMOylate Target Proteins R-HS...   
 12  Reactome_2022            

In [39]:
miRNA_targets_filtered.keys()

dict_keys(['miR-hsa-miR-374a-5p', 'miR-hsa-miR-4293', 'miR-hsa-miR-4291', 'miR-hsa-miR-194-5p', 'miR-hsa-miR-4294', 'miR-hsa-miR-29a-3p', 'miR-hsa-miR-4296', 'miR-hsa-miR-4298', 'miR-hsa-miR-429', 'miR-hsa-miR-330-5p', 'miR-hsa-miR-4295', 'miR-hsa-miR-424-5p', 'miR-hsa-miR-135b-5p', 'miR-hsa-miR-155-3p', 'miR-hsa-miR-4292', 'miR-hsa-miR-16-5p', 'miR-hsa-miR-141-3p', 'miR-hsa-miR-335-5p', 'miR-hsa-miR-27b-3p', 'miR-hsa-miR-29c-3p', 'miR-hsa-miR-4290', 'miR-hsa-miR-27a-3p', 'miR-hsa-miR-200b-3p', 'miR-hsa-miR-4299', 'miR-hsa-miR-4297', 'miR-hsa-miR-21-5p'])

In [40]:
import pickle

with open('dict_of_dfs.pickle', 'wb') as f:
    pickle.dump(result_dict, f)

In [41]:
import pandas as pd

# Create an Excel writer object
writer = pd.ExcelWriter('miR_enr_12hrs.xlsx')

# Iterate over the dictionary items
for key, value in result_dict.items():
    # Save each nested table as a separate sheet in the Excel file
    value.to_excel(writer, sheet_name=key, index=False)

# Save the Excel file
writer.save()


/var/folders/7h/_mrm1yh5299bvqql6k199xb00000gn/T/ipykernel_13625/3669307579.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
